# Criteo Click Through Prediction using ThirdAI's Universal Deep Transformer(UDT) APIs
This notebook shows how to build a Click Through Prediction model using ThirdAI's UDT.

In [ ]:
!pip3 install thirdai --upgrade

# Download and Process Dataset

We will now create a sample dataset for training. Criteo-Click Logs dataset can be downloaded from https://ailab.criteo.com. Once datasets are downloaded convert the datasets into csv format. 

Column headers: `<label>` is titled as `<label>`, `<integer feature 1>` is titled as `numeric_1` and `<categorical feature 1>` as `cat_1`

<span style="color:red">Disclaimer: Remember that the following model is trained on sample data, to achieve better AUC train on the original criteo-dataset.</span>

In [ ]:
import numpy as np
import pandas as pd

TRAIN_FILE = 'train_file.txt'
TEST_FILE = 'test_file.txt'

header = 'label,numeric_1,numeric_2,numeric_3,numeric_4,numeric_5,numeric_6,numeric_7,numeric_8,numeric_9,numeric_10,numeric_11,numeric_12,numeric_13,'
header += 'cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,'
header += 'cat_22,cat_23,cat_24,cat_25,cat_26'

column_header=header.split(",")

num_training_data=1200
num_numeric_features=13
num_categorical_features=26

numeric_data = np.random.randint(1500, size=(num_training_data, num_numeric_features))
categorical_data = np.random.randint(10000, size=(num_training_data, num_categorical_features))
label = np.random.randint(2, size=(num_training_data, 1))

data_X , data_Y = np.concatenate((numeric_data, categorical_data), axis=1), label

train_data = np.concatenate((data_Y[:1000], data_X[:1000]), axis=1)
train_df = pd.DataFrame(train_data, columns=column_header)
train_df.to_csv(TRAIN_FILE, index=False)


test_data = np.concatenate((data_Y[1000:], data_X[1000:]), axis=1)
train_df = pd.DataFrame(test_data, columns=column_header)
train_df.to_csv(TEST_FILE, index=False)

train_df.head(10)

# UDT Initialization


We can create a UDT model specific for Criteo Click Through Prediction as follows. Here we define Bolt Datatype for all the columns, specify the target, n_target classes and the embedding dimension for UDT. 

In [ ]:
from thirdai import bolt
import numpy as np
from sklearn.metrics import roc_auc_score


tabular_model = bolt.UniversalDeepTransformer(
    data_types={
        "numeric_1": bolt.types.numerical(range=(0, 1500)),
        "numeric_2": bolt.types.numerical(range=(0, 1500)),
        "numeric_3": bolt.types.numerical(range=(0, 1500)),
        "numeric_4": bolt.types.numerical(range=(0, 1500)),
        "numeric_5": bolt.types.numerical(range=(0, 1500)),
        "numeric_6": bolt.types.numerical(range=(0, 1500)),
        "numeric_7": bolt.types.numerical(range=(0, 1500)),
        "numeric_8": bolt.types.numerical(range=(0, 1500)),
        "numeric_9": bolt.types.numerical(range=(0, 1500)),
        "numeric_10": bolt.types.numerical(range=(0, 1500)),
        "numeric_11": bolt.types.numerical(range=(0, 1500)),
        "numeric_12": bolt.types.numerical(range=(0, 1500)),
        "numeric_13": bolt.types.numerical(range=(0, 1500)),
        "cat_1": bolt.types.categorical(),
        "cat_2": bolt.types.categorical(),
        "cat_3": bolt.types.categorical(),
        "cat_4": bolt.types.categorical(),
        "cat_5": bolt.types.categorical(),
        "cat_6": bolt.types.categorical(),
        "cat_7": bolt.types.categorical(),
        "cat_8": bolt.types.categorical(),
        "cat_9": bolt.types.categorical(),
        "cat_10": bolt.types.categorical(),
        "cat_11": bolt.types.categorical(),
        "cat_12": bolt.types.categorical(),
        "cat_13": bolt.types.categorical(),
        "cat_14": bolt.types.categorical(),
        "cat_15": bolt.types.categorical(),
        "cat_16": bolt.types.categorical(),
        "cat_17": bolt.types.categorical(),
        "cat_18": bolt.types.categorical(),
        "cat_19": bolt.types.categorical(),
        "cat_20": bolt.types.categorical(),
        "cat_21": bolt.types.categorical(),
        "cat_22": bolt.types.categorical(),
        "cat_23": bolt.types.categorical(),
        "cat_24": bolt.types.categorical(),
        "cat_25": bolt.types.categorical(),
        "cat_26": bolt.types.categorical(),
        "label": bolt.types.categorical(),
    },
    target="label",
    n_target_classes=2,
    options={"embedding_dimension": 512},
)

# Training

We will now train the UDT with just one line of code. Here we are specifying the training file name, number of epochs to train and maximum number of batches we want to load in memory at once. `max_in_memory_batches` support loaing data in a streaming fashion. You can change `max_in_memory_batches` based on your own memory specification.

In [ ]:


tabular_model.train(filename=TRAIN_FILE, epochs=1, max_in_memory_batches=1000)



# Evaluation

Evaluating the UDT is also just one line of code. Evaluate function by default returns activations. Then, we are using sklearn's roc_auc_score to calculate roc_auc_score for the model we trained.

In [ ]:
activations = tabular_model.evaluate(filename=TEST_FILE, metrics=["categorical_accuracy"])

true_labels = np.zeros(activations.shape[0], dtype=np.float32)
with open(TEST_FILE) as f:
    header = f.readline()
    count = 0
    for line in f:
        true_labels[count] = np.float32(line.split(",")[0])
        count += 1

roc_auc = roc_auc_score(true_labels, activations[:, 1])

print("ROC_AUC:", roc_auc)



In [ ]:
import os
os.remove(TRAIN_FILE)
os.remove(TEST_FILE)